In [1]:
%matplotlib qt
from __future__ import print_function

import rospy
import rosbag
import itertools
from  tqdm import tqdm
from geometry_msgs.msg import PoseWithCovarianceStamped
import tf
import numpy as np

import matplotlib.pyplot as plt
import os

%load_ext autoreload
%autoreload 2

In [2]:
# Robot axis directions
#                   ┌───────┐         
#                   │ lidar │                                           
#        ╔══════════╧═══════╧═════════╗                 ┌─┐    |    ┌─┐
#      ┌─╢                            ║                 │6│    |    │3│                 
#   [3]│ ║                            ║                 └─┘    |    └─┘        
#      └─╢                            ║                 ┌─┐    |    ┌─┐ 
#      ┌─╢            z(+)            ║                 │5│    |    │2│  
#   [2]│ ║              │             ║                 └─┘    |    └─┘
#      └─╢        x(+)──┘             ║                 ┌─┐    |    ┌─┐
#      ┌─╢                            ║                 │4│    |    │1│
#   [1]│ ║                            ║                 └─┘  z(+)   └─┘   
#      └─╢                            ║                        │           
#        ╚════════════════════════════╝                        └── y(+) 
                                         
mag_loc = { '/imu1/mag': {'x':0.34, 'y': 0.09, 'z':0.21},
            '/imu2/mag': {'x':0.34, 'y': 0.09, 'z':0.34},
            '/imu3/mag': {'x':0.34, 'y': 0.09, 'z':0.48},
            '/imu4/mag': {'x':0.34, 'y':-0.09, 'z':0.21},
            '/imu5/mag': {'x':0.34, 'y':-0.09, 'z':0.34},
            '/imu6/mag': {'x':0.34, 'y':-0.09, 'z':0.48},
          }
mag_loc

{'/imu1/mag': {'x': 0.34, 'y': 0.09, 'z': 0.21},
 '/imu2/mag': {'x': 0.34, 'y': 0.09, 'z': 0.34},
 '/imu3/mag': {'x': 0.34, 'y': 0.09, 'z': 0.48},
 '/imu4/mag': {'x': 0.34, 'y': -0.09, 'z': 0.21},
 '/imu5/mag': {'x': 0.34, 'y': -0.09, 'z': 0.34},
 '/imu6/mag': {'x': 0.34, 'y': -0.09, 'z': 0.48}}

In [3]:
## Calibration     ┌                  ┐
# Mag_calibrated = │ (r/sx)(mag_x-bx) │ 
#                  │ (r/sy)(mag_y-by) │
#                  │ (r/sz)(mag_z-bz) │
#                  └                  ┘


#recalibration
#b = {'/imu1/mag': -0.2663792834412799,
#     '/imu2/mag': -0.3184230838292076,
#     '/imu3/mag': -0.7266093876552278,
#     '/imu4/mag': -0.6947709287264497,
#     '/imu5/mag': -0.6560786531514198,
#     '/imu6/mag': -0.5677440295570412}

def read_spec(filename):
    try:
        with open(filename) as f:
            calibration = {}
            for line in f:
                if 'x axis span' in line: calibration['sx'] = float(line.split()[-1]); continue
                if 'y axis span' in line: calibration['sy'] = float(line.split()[-1]); continue
                if 'z axis span' in line: calibration['sz'] = float(line.split()[-1]); continue
                if 'x position'  in line: calibration['bx'] = float(line.split()[-1]); continue
                if 'y position'  in line: calibration['by'] = float(line.split()[-1]); continue
                if 'z position'  in line: calibration['bz'] = float(line.split()[-1]); continue
                if 'magnification' in line: calibration['r'] = float(line.split()[-1]); continue
            return calibration
    except:
        print('[Error] Could not read_spec file {:s}'.format(filename))

In [ ]:
mag_calibration = {}
for imu_number in range(1,7):
    imu = 'imu{:d}'.format(imu_number)
    topic_name = '/{:s}/mag'.format(imu)
    mag_calibration[topic_name] = read_spec('spec/correct_file_nict{:s}.spec'.format(imu))
#mag_calibration

In [ ]:
import rosbag
import tf
import numpy as np
import pandas as pd
import time

from sensor_msgs.msg import MagneticField

def get_mag(map_name,Pose,t_offset,mag_calibration=None):
    """
    load all magnetic data from rosbag
    """
    sensors_bag_filename = '{0:s}/{1:s}/rosbag/{1:s}_sensors.bag'.format(root_folder,map_name)
    print('Reading {:60s}'.format(sensors_bag_filename),end='')
    sensors_bag = rosbag.Bag(sensors_bag_filename)
    print('\t Done')
  
    if mag_calibration is not None: calibrate_data = True

    sensors_topics = ['/imu{:d}/mag'.format(i) for i in range(1,7)]
    print(sensors_topics); time.sleep(1)

    sensors = {topic:list() for topic in sensors_topics}

    Ti,Xb,Yb,X,Y,Z,Yaw,Topic,MagX,MagY,MagZ = [list() for i in range(11)]

    for topic, msg, t in tqdm(sensors_bag.read_messages(topics=sensors_topics)):
        ti = msg.header.stamp.secs-t_offset + 1e-9*msg.header.stamp.nsecs
        # interpolate pose
        yaw_c = np.interp(ti,Pose[:,0],Pose[:,3])
        yaw_s = np.interp(ti,Pose[:,0],Pose[:,4])
        yaw   = np.arctan2(yaw_s,yaw_c)
        x_body = np.interp(ti,Pose[:,0],Pose[:,1])
        y_body = np.interp(ti,Pose[:,0],Pose[:,2])
        # consider mag location
        x_mag = x_body+mag_loc[topic]['x']*np.cos(yaw)+mag_loc[topic]['y']*np.sin(yaw)
        y_mag = y_body-mag_loc[topic]['x']*np.sin(yaw)+mag_loc[topic]['y']*np.cos(yaw)
        z_mag = mag_loc[topic]['z']
        # raw data
        mag_x = msg.magnetic_field.x/100.
        mag_y = msg.magnetic_field.y/100.
        mag_z = msg.magnetic_field.z/100.
        if calibrate_data:
            mag_x = (mag_calibration[topic]['r']/mag_calibration[topic]['sx'])*(mag_x-mag_calibration[topic]['bx'])
            mag_y = (mag_calibration[topic]['r']/mag_calibration[topic]['sy'])*(mag_y-mag_calibration[topic]['by'])
            mag_z = (mag_calibration[topic]['r']/mag_calibration[topic]['sz'])*(mag_z-mag_calibration[topic]['bz'])
        # TODO: filter data

        # add datapoint
        Ti.append(ti)
        X.append(x_mag)
        Y.append(y_mag)
        Z.append(z_mag)
        Xb.append(x_body)
        Yb.append(y_body)
        Yaw.append(yaw)
        Topic.append(topic)
        MagX.append(mag_x)
        MagY.append(mag_y)
        MagZ.append(mag_z)

    df = pd.DataFrame.from_dict({
            'time':Ti,
            'x':X,
            'y':Y,
            'z':Z,
            'xb':Xb,
            'yb':Yb,
            'yaw':Yaw,
            'topic':Topic,
            'mag_x':MagX,
            'mag_y':MagY,
            'mag_z':MagZ
            })
    
    # correct for bias
    robot_bias = dict()
    for topic,di in df[:1000].groupby('topic',as_index=False):
        robot_bias[topic] = di['mag_z'].mean()
    # Compute average from all sensors
    mag_avg = np.mean([v for k,v in robot_bias.items()])
    for k,v in robot_bias.items():
        robot_bias[k] = v - mag_avg
    # Correct robot bias
    for topic,di in df.groupby('topic',as_index=False):
        df.loc[di.index,'mag_z'] -= robot_bias[topic] 
    
    return df

def filter_mag(df):
    # create time-slices
    ts = 0.1
    time_slices = np.arange(df["time"].min(),df["time"].max(),ts)
    tmp = df.groupby(pd.cut(df["time"],time_slices))
    df_list = [group for name, group in tmp]
    
    XY = np.asarray([[di['xb'].mean(), di['yb'].mean()] for di in df_list])
    print('# time slices',len(df_list))
    time.sleep(0.2)
    
    # distance filter
    d_thr = 0.2 # merge if less than 50cm
    Dxy = list()
    i = 0
    c = 0
    for c in tqdm(range(len(df_list))):
        dxy = (df_list[i]['xb'].max()-df_list[i]['xb'].min())**2+(df_list[i]['yb'].max()-df_list[i]['yb'].min())**2
        if dxy < d_thr**2:
            #print(c,'merge')
            try:
                df_list[i] = pd.concat([df_list[i],df_list.pop(i+1)])
            except:
                pass
        else:
            i +=1
            Dxy.append(dxy**.5)

    return df_list

def create_mag_dataset(df_list):
    XYZ = list()
    MZ  = list()
    for df in tqdm(df_list):
        for topic,di in df.groupby('topic',as_index=False):
            XYZ.append([di['x'].mean(), di['y'].mean(), di['z'].mean()])
            MZ.append([di['mag_z'].mean()])

    XYZ = np.asarray(XYZ).astype(np.float32)
    MZ  = np.asarray(MZ).astype(np.float32)
    mz = np.mean(MZ).astype(np.float32)
    MZ = MZ-mz
    return XYZ,MZ,mz

# Example use

In [ ]:
root_folder = '/mnt/matrix/rosbag/processed_data/nict'
all_maps = 'udai-01'

print('\n####################\n  map: {:s} \n####################'.format(map_name)); time.sleep(0.2)
Pose, t_offset = create_gt(map_name)
print('Pose: {:d}'.format(int(Pose.shape[0])))
print('>> Get Mag'); time.sleep(0.2)
df = get_mag(map_name,Pose,t_offset,mag_calibration=mag_calibration)
print('>> Filter time/distance'); time.sleep(0.2)
df_list = filter_mag(df)
print('>> Compute Mag dataset'); time.sleep(0.2)
XYZ,MZ,mz = create_mag_dataset(df_list)

print('Mag  : ',MZ.shape,
      '\nXYZ  : ',XYZ.shape)

f,ax = plt.subplots()
ax.scatter(XYZ[:,0],XYZ[:,1])
ax.set_aspect('equal')

# Save data
#try:
#    os.mkdir('{0:s}/{1:s}/invisible-maps/'.format(root_folder,map_name))
#except:
#    pass
#save_filename = '{0:s}/{1:s}/invisible-maps/{1:s}_MagzData.npz'.format(root_folder,map_name)
#np.savez_compressed(save_filename,XYZ=XYZ,MZ=MZ,mz_offset=mz)